In [37]:
import pandas as pd
import os
import glob
from ast import literal_eval

In [4]:
result_dir = "../data/relation_extraction/qcodes"

In [31]:
property_dir = "../data/relation_extraction/properties"

In [59]:
df_pih = pd.read_csv( f"{result_dir}/pih.csv")

In [63]:
def strech_dfs(file ,outputfile):
    df= pd.read_csv(file)
    props=[]
    e1s =[]
    e2s=[]
    sentences=[]
    for prop, e1, e2, sentence in zip(df["Property"], df["e1"], df["e2"], df["Sentence"]):
        if "," in prop:
            prop1,prop2 = prop.split(",")
            props.append(prop1)
            props.append(prop2)
            e1s.append(e1)
            e1s.append(e1)
            e2s.append(e2)
            e2s.append(e2)
            sentences.append(sentence)
            sentences.append(sentence)
        else:
            props.append(prop)
            e1s.append(e1)
            e2s.append(e2)
            sentences.append(sentence)
    df_new = pd.DataFrame.from_dict({
        "Property":props,
        "e1":e1s,
        "e2":e2s,
        "Sentence":sentences
        })
    df_new.to_csv(outputfile, index=False)
    

In [64]:
for lang in ["bi", "cbk-zam", "ht", "jam", "pih", "tpi"]:
    file = f"{result_dir}/{lang}.csv"
    outputfile = f"../data/relation_extraction/streched_df/{lang}.csv"
    strech_dfs(file ,outputfile)

In [77]:
def is_triple(prop, prop_list):
    try:
        if prop in prop_list:
            return True
        else:
            return False
    except Exception:
        return False

In [80]:
def merge_files(lang):
    df_prop = pd.read_csv( f"{property_dir}/{lang}.csv", sep="\t")
    df= pd.read_csv( f"../data/relation_extraction/streched_df/{lang}.csv")
    df_prop["Properties"]=df_prop["Properties"].map(literal_eval, na_action="ignore")
    df_prop =df_prop.rename(columns={"ENT1":"e1", "ENT2":"e2"})
    df_merge = pd.merge(df, df_prop, how="left", on=["e1", "e2"])
    df_merge["TripleInWD"]= df_merge[["Property", "Properties"]].apply(lambda x: is_triple(*x), axis=1)
    df_merge.to_csv(f"../data/relation_extraction/results/{lang}.csv", index=False)
    

In [81]:
for lang in ["bi", "cbk-zam", "ht", "jam", "pih", "tpi"]:
    merge_files(lang)

In [2]:
data_dir = "../data/creoles_relation_classification_pre_verified/"

In [30]:
for lang in ["bi", "cbk-zam", "ht", "jam", "pih", "tpi"]:
    print(lang)
    df_input = pd.read_csv(f"{data_dir}2023-04-24-{lang}.csv", sep="|")
    input_sentences = df_input["Sentence"].to_list()
    df_output = pd.read_csv(f"{result_dir}/{lang}.csv")
    output_sentences = df_output["Sentence"].tolist()
    print(set(input_sentences).difference(set(output_sentences)))
    print(set(output_sentences).difference(set(input_sentences)))


bi
set()
set()
cbk-zam
set()
set()
ht
{'[[Somon se yon [[pwason]].', '[[Lisbòn]] se [[kapital]] [[Pòtigal]].'}
{'[[Somon]] se yon [[pwason]].', '[[Lisbòn]] se kapital [[Pòtigal]].'}
jam
{'[[Matamala]] de Almazán a wahn miunisipaliti luokiet ina di pravins a [[Soria]], Kiastiil ahn León, Spien.', '[[Aluuminom]] a wah [[kimikal eliment]] ina di [[buoran]] gruup wid simbal Al ah atamik nomba 13.'}
{'[[Matamala de Almazán]] a wahn miunisipaliti luokiet ina di pravins a [[Soria]], Kiastiil ahn León, Spien.', '[[Aluuminom]] a wah [[kimikal eliment]] ina di buoran gruup wid simbal Al ah atamik nomba 13.'}
pih
{'[[Abel Askoena]] (Ieprel 1, 1988, [[Madrid]]; fuul naem esa Abel Askoena Markuus) esa wun Spanish [[akter]].'}
{'[[Abel Askoena]] (Ieprel 1, 1988, Madrid; fuul naem esa Abel Askoena Markuus) esa wun Spanish [[akter]].'}
tpi
{'[[Vilnius]] i [[kapitol]] bilong [[Lituwenia]].', '[[Toomas Hendrik]] Ilves i [[presiden]] bilong Estonia.', '[[Sofia]] em i [[kapitol]] long kantri [[Balgeria]].

In [27]:
def stretch_data(df):
    properties = df["Property"].tolist()
    properties = [x.split(",") if "," in x else x for x in properties ]
    

In [25]:
for file in glob.glob(f"{result_dir}/**.csv"):
    print(file)
    df = pd.read_csv(file)
    df["same"]= df["e1"]==df["e2"]
    
    print(len(df), len(df.dropna()))
    
    print(df[df["same"]==True])

../data/relation_extraction/qcodes/jam.csv
99 99
Empty DataFrame
Columns: [Property, e1, e2, Sentence, same]
Index: []
../data/relation_extraction/qcodes/ht.csv
98 98
Empty DataFrame
Columns: [Property, e1, e2, Sentence, same]
Index: []
../data/relation_extraction/qcodes/bi.csv
100 100
Empty DataFrame
Columns: [Property, e1, e2, Sentence, same]
Index: []
../data/relation_extraction/qcodes/cbk-zam.csv
100 100
Empty DataFrame
Columns: [Property, e1, e2, Sentence, same]
Index: []
../data/relation_extraction/qcodes/pih.csv
99 99
Empty DataFrame
Columns: [Property, e1, e2, Sentence, same]
Index: []
../data/relation_extraction/qcodes/tpi.csv
98 98
Empty DataFrame
Columns: [Property, e1, e2, Sentence, same]
Index: []


In [15]:

for file in glob.glob(f"{data_dir}**_output.csv"):
    lang = file.replace("../data/creoles_relation_classification_pre_verified/2023-04-24-", "").replace("_output.csv","")
    print(lang)
    df = pd.read_csv(file)
    df["e1"] = df["e1"].str.replace("QQ", "Q")
    df["e2"] = df["e2"].str.replace("QQ", "Q")
    print(len(df), len(df.dropna()))
    df.to_csv(f"{result_dir}/{lang}.csv", index=False)

jam
99 99
bi
100 100
tpi
98 97
pih
99 91
cbk-zam
100 100
ht
98 98


In [7]:
df = pd.read_csv("../data/creoles_relation_classification_pre_verified/2023-04-24-jam_output.csv")

In [8]:
df

,Property,e1,e2,Sentence
0,P36,QQ45,QQ597,"Di kiapital a [[Puotigal]] niem ""[[Lizban]]""."
1,P1376,QQ472,QQ219,[[Sofiya]] a di kiapital fi [[Bulgieria]].
2,P1376,QQ239,QQ31,[[Broslz]] a di kiapital fi [[Beljiom]].
3,P1376,QQ220,QQ38,[[Ruom]] a di kyapital fi [[Itali]].
4,P1376,QQ649,QQ159,[[Masko]] a di kyapital fi [[Rosha]].
...,...,...,...,...
94,P31,QQ682,Q11344,[[Solfa]] a di [[kimikal eliment]] wid simbal ...
95,P31,QQ627,Q11344,[[Naichojen]] a wah [[kimikal eliment]] wid si...
96,P31,QQ629,Q11344,[[Axijen]] a wah [[kimikal eliment]] wid simba...
97,P31,QQ556,Q11344,[[Aijrojin]] a wah [[kimikal eliment]] wid kim...


In [11]:
df["e1"] = df["e1"].str.replace("QQ", "Q")

In [12]:
df["e2"] = df["e2"].str.replace("QQ", "Q")

In [14]:
df.dropna()

,Property,e1,e2,Sentence
0,P36,Q45,Q597,"Di kiapital a [[Puotigal]] niem ""[[Lizban]]""."
1,P1376,Q472,Q219,[[Sofiya]] a di kiapital fi [[Bulgieria]].
2,P1376,Q239,Q31,[[Broslz]] a di kiapital fi [[Beljiom]].
3,P1376,Q220,Q38,[[Ruom]] a di kyapital fi [[Itali]].
4,P1376,Q649,Q159,[[Masko]] a di kyapital fi [[Rosha]].
...,...,...,...,...
94,P31,Q682,Q11344,[[Solfa]] a di [[kimikal eliment]] wid simbal ...
95,P31,Q627,Q11344,[[Naichojen]] a wah [[kimikal eliment]] wid si...
96,P31,Q629,Q11344,[[Axijen]] a wah [[kimikal eliment]] wid simba...
97,P31,Q556,Q11344,[[Aijrojin]] a wah [[kimikal eliment]] wid kim...
